In [2]:
import numpy as np
import gurobi 
# as gb
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from numpy.random import normal, randint
import tensorflow as tf


GUROBI_TIMEOUT = 5 # in second

%matplotlib inline


# Helper Function

In [1]:
# Helper Functions to solve Gurobi Models

class Gurobi_Model():
    '''
        Class Method I made to help me encapsulate the whole Gurobi model creation, 
        Finding Slack Variable and perform sensitivity analysis
    '''
    
    def __init__(self, obj_linear, cons, sense, b, variable_types = 'C',
                lower_bound = None, upper_bound = None,
                optimisation_type = gurobi.GRB.MAXIMIZE, obj_quadratic = None) -> None:
        '''
            Initialising the Model
        '''
        if (obj_quadratic is None) and (obj_linear is None):
            raise Exception("No objective defined")

        constraints, variables = cons.shape

        ## Optimisation Model at work (Maximise)
        try:
            self.model = gb.Model()
            self.model_X = self.model.addMVar(variables,
                                        vtype=variable_types,
                                        lb= lower_bound, ub= upper_bound)

            self.model_constraints = self.model.addMConstr(cons, self.model_X, sense, b)
            self.model.setMObjective(obj_quadratic, obj_linear, 0, sense=optimisation_type)
            self.model.Params.OutputFlag = 0
            self.model.Params.TimeLimit = GUROBI_TIMEOUT

            self.model.optimize()

        except Exception:
            print("Error in optimising")
            raise Exception
        
        ## Assigning stuff for other use cases
        try:
            self.obj_q = obj_quadratic
            self.obj = obj_linear
            self.constraint = cons
            self.sense = sense
            self.b = b
        except Exception:
            print("Error in Storing equations")
            raise Exception
        return

    @property
    def optimal_obj(self): return self.model.objVal

    @property
    def optimal_x(self): return self.model_X.x

    @property
    def output(self):
        return {'objVal': self.optimal_obj,
                'x': self.optimal_x}

NameError: name 'gurobi' is not defined

# Q1 Neural Nets

In [ ]:
# problem 1

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 #normalize the data, 8bit pixel representation to currently between 0 to 255

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]

print(x_train.shape, x_test.shape, y_test.shape, y_train.shape, ndata_train, ndata_test) 

xshape = x_train.shape[1:4]
xshape

In [ ]:
NNmodel = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=5,kernel_size=(5,5), activation= 'relu',input_shape=xshape),
        tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides=2),
        tf.keras.layers.Conv2D(filters=10,kernel_size=(6,6),activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l1(0.005)),
        tf.keras.layers.Dense(32,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(10,activation=tf.nn.softmax)
        ])

In [ ]:
NNmodel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

NNmodel.summary()

In [ ]:
NNmodel.fit(x_train,y_train,epochs=5,validation_split=0.4,batch_size=200)

In [ ]:
pred_probs = NNmodel.predict(x_test)
pred = np.argmax(pred_probs, axis=1)
print(np.mean(pred==y_test))

## Answers

- a) Number of nodes in the preceeding layer: 160
- b) Total number of trainable parameters: 27256
- c) Number of Gradient descent calculations: (Number of batches per epoc * Number of epocs) 1250
- d) Test Set Accuracy: 0.1

# Q2 Newsvendor Problem

$$\max_{q} \text{ Profit}_i = \frac{1}{N}\sum_{i=1}^{N} {(pD_i - qc - g(D_i - q)^+ - t(q - D_i)^+)}$$

where:
- $(x)^+ = \max (x,0)$
- $D_i$ is the demand on Day _i_
- $q$ is the **Quantity** produced initially
- $p$ is the **Price** (Sale Price)
- $c$ is the **Cost** of production (Cost Price)
- $g$ is the **Expedited Cost** of production
- $t$ is the **Disposal** Cost

Thus the NLP can be reformulated as an LP in the following way:

$$\max_{q,h_1, h_2, ..., h_N} \text{ Profit}_i = \frac{1}{N}\sum_{i=1}^{N} {h_i}$$
subject to:
$$
h < {(pD_i - cq - g(D_i - q))} \\
h < {(pD_i - cq - t(q - D_i))} \\
h > -\infty \\
q > 0
$$

upon rewriting, we get:

$$\max_{q,h_1, h_2, ..., h_N} \text{ Profit}_i = \frac{1}{N}\sum_{i=1}^{N} {h_i}$$
subject to:
$$
h + (c-g)q < (p-g)D_i \\
h + (c+t)q < (p+t)D_i \\
h > -\infty \\
q > 0
$$



<!--Here Quantity, Cost and sale price are all vectors $\epsilon\text{  } \R^{5x1}$-->

In [2]:
cost = 2.0
g =  3.0

price = 3.75
t = 0 #0.15   -No disposal cost

In [3]:
demand = pd.read_csv('demand130.csv')[['data_point','demand']]

num_days = demand.shape[0]
print(demand.shape)
demand.head()

NameError: name 'pd' is not defined

In [1]:
# decision variables are (q, h1, h2, ..., h_num_days)
obj = np.zeros(num_days+1)
obj[1:] = 1.0/num_days

lb = np.zeros(num_days+1)
lb[1:] = -np.inf # quantity printed needs to be non-negative, but profit on a given day could possibly be negative

rhs = np.zeros(2*num_days)

direction = np.array(['<']*(2*num_days))

var_type = np.array(['I']+['C']*num_days)

NameError: name 'np' is not defined

In [ ]:
A = np.zeros((2*num_days,num_days+1))

for r in range(num_days):
    A[2*r ,[0,r+1]] = [cost - g,1] # location 0 is q, location r+1 is this h
    rhs[2*r] = (price - g)*demand.demand.iloc[r]

    A[2*r+1 ,[0,r+1]] = [cost + t,1]
    rhs[2*r+1] = (price + t)*demand.demand.iloc[r]
A.shape

(200, 101)

In [2]:
newsvendor_model = Gurobi_Model(obj_linear= obj, 
                        cons= A, 
                        sense= direction, 
                        b= rhs,
                        lower_bound= lb,
                        variable_types= var_type
                    )
newsvendor_model.output

NameError: name 'Gurobi_Model' is not defined

In [ ]:
print(f'Number of Newspapers to print: {newsvendor_model.optimal_x[0]}')

##  Answers

a) total constraints : 200

b) decision variables : 101

c) Number of decision variables with lower bound of 0 : 1

d) Number of Newspapers to print : 1191

* Max profit at : $ 2065.095

# Q3 - Simulation

In [2]:
def concert_admission():
    people_in_line = 517
    time_left = 3600 # 1 hour left (in seconds)
    while (people_in_line > 0):
        # check one person
        security_time = np.random.choice([6,7,8])
        time_left -= security_time
        people_in_line -= 1
        if time_left <=0:
            break
    return people_in_line

In [6]:
nsim = 10000
people_left = np.zeros(nsim)
for simulation in tqdm(range(nsim)):
    people_left[simulation] = concert_admission()
people_left

100%|██████████| 10000/10000 [00:47<00:00, 212.20it/s]


array([3., 2., 5., ..., 6., 6., 0.])

In [7]:
print(f'P( At least one person is turned away: {np.round(np.mean(people_left > 0), 2)}')

P( At least one person is turned away: 0.76


## Answers

a) choice 3, 4 
  - np.random.choice([6,7,8])
  - np.random.choice(3) + 6

b) 2 - Fewer (because Probability security check takes more time is higher)

c) around 0.75
